# 🚀 CLI interaction with Python netmiko

Welcome to your first step into network automation with Python & Netmiko!</br>
This tutorial shows how to connect to a Cisco IOS-XR router, read configurations, make small safe changes, and truly see why scripting beats manual CLI.

🧰 Prerequisites

- 🐍 Python 3.9+
- 🔌 SSH access to your Cisco IOS-XR device. We are using the [Always-On Cisco IOSXR from DevNet Sandboxes](https://devnetsandbox.cisco.com/DevNet/catalog/ios-xr-always-on_ios-xr-always-on#instructions)
- 👤 `.env` file with its credentials and URL (available in this repository)

🔁 First of all, create a virtual environment with the following commands:

In [ ]:
!python3 -m venv .venv && source .venv/bin/activate
!pip install -r requirements.txt

✅ Let's import now our libraries and apply our environment variables from the `.env` file:

In [3]:
import os
import difflib
import datetime
from dotenv import load_dotenv
from netmiko import ConnectHandler

load_dotenv()

True

🤖 This is the definition of our device in Netmiko using the environment variables: 

In [4]:
device = {
    "device_type": "cisco_xr",  # important: IOS-XR
    "host": os.getenv("XR_HOST"),
    "username": os.getenv("XR_USER"),
    "password": os.getenv("XR_PASS"),
    "fast_cli": True,           # faster reads
}

### 1️⃣ Read current configurations of the devices
Let's connect to the router and runs harmless show commands like:

- show version
- show inventory
- show ipv4 interface brief

💡 Why it matters:

- 🔍 Perfect sanity check before any config changes
- 🧘‍♀️ Safe: completely non-intrusive
- ⚡ Faster than manually logging in 5 routers to check the same info

In [14]:
with ConnectHandler(**device) as conn:
    print("Connected:", conn.find_prompt())

    # A couple of safe show commands
    ver = conn.send_command("show version")
    inv = conn.send_command("show inventory | utility egrep -v 'SN:|VID:'")
    int_brief = conn.send_command("show ipv4 interface brief")

    print("\n=== show version ===\n", ver)
    print("\n=== show inventory (condensed) ===\n", inv)
    print("\n=== show ipv4 interface brief ===\n", int_brief)

Connected: RP/0/RP0/CPU0:ios#

=== show version ===
 
Tue Nov  4 11:11:32.937 UTC
Cisco IOS XR Software, Version 7.3.2
Copyright (c) 2013-2021 by Cisco Systems, Inc.

Build Information:
 Built By     : ingunawa
 Built On     : Wed Oct 13 20:00:36 PDT 2021
 Built Host   : iox-ucs-017
 Workspace    : /auto/srcarchive17/prod/7.3.2/xrv9k/ws
 Version      : 7.3.2
 Location     : /opt/cisco/XR/packages/
 Label        : 7.3.2-0

cisco IOS-XRv 9000 () processor
System uptime is 2 minutes


=== show inventory (condensed) ===
 
Tue Nov  4 11:11:33.524 UTC
NAME: "0/0", DESCR: "Cisco IOS-XRv 9000 Centralized Line Card"

NAME: "0/RP0", DESCR: "Cisco IOS-XRv 9000 Centralized Route Processor"

NAME: "Rack 0", DESCR: "Cisco IOS-XRv 9000 Centralized Virtual Router"


=== show ipv4 interface brief ===
 
Tue Nov  4 11:11:34.278 UTC

Interface                      IP-Address      Status          Protocol Vrf-Name
Loopback100                    1.1.1.100       Up              Up       default 
Loopback555 

### 2️⃣ Create a snapshot of the current configurations

Let's take a snapshot of the running configuration and save it locally with a timestamp in the folder `backup/`.

💡 Why it matters:

- 🛡️ Disaster recovery made easy
- 🕰️ Every backup is timestamped for version tracking
- 📋 Great for audits and compliance proofs

> *“Never automate without a backup”* — every senior network engineer ever.

In [5]:
ts = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

outfile = f"backups/{os.getenv('XR_HOST')}-{ts}.running-config.txt"
os.makedirs("backups", exist_ok=True)

with ConnectHandler(**device) as conn:
    running = conn.send_command("show running-config")
    with open(outfile, "w") as f:
        f.write(running)

print("Backup saved to:", outfile)

Backup saved to: backups/sandbox-iosxr-1.cisco.com-20251104-105731.running-config.txt


### 3️⃣ Changing and committing configurations

Let's create a new loopback interface safely — with full diff visibility.

The workflow is the following:
✅ pre-change snapshot → 🧪 dry-run diff → 🚀 apply config → 💾 commit → 🔍 verify diff.

💡 What happens here

1. Connects to your Cisco IOS-XR router via SSH
2. Checks if Loopback201 exists and captures its current config
3. Shows a dry-run diff before any change is made
4. Prompts you for confirmation
5. Creates or updates interface Loopback201 with a description
6. Performs an explicit commit (required on IOS-XR)
7. Retrieves and displays a post-change diff to verify what changed

In [ ]:
commands = [
    "interface Loopback201",
    "description TESTDEMO"
]

# Connect to the device
with ConnectHandler(**device) as conn:
    print("✅ Connected to:", conn.find_prompt())

    # Step 1: Read current (pre-change) configuration
    before_config = conn.send_command("show running-config interface Loopback201")

    # Step 2: Generate the intended config as text
    intended_config = "\n".join(commands)

    # Step 3: Display a "dry-run" diff
    print("\n=== 🧪 Dry-run diff (before → intended) ===")
    diff = difflib.unified_diff(
        before_config.splitlines(),
        intended_config.splitlines(),
        fromfile="before",
        tofile="intended",
        lineterm=""
    )
    diff_text = "\n".join(diff)
    print(diff_text if diff_text else "(no differences detected — interface may already match intended state)")

    # Step 4: Apply the candidate config
    print("\n=== 🚀 Applying candidate configuration ===")
    output = conn.send_config_set(commands)
    print(output)

    # Step 5: Commit explicitly on IOS-XR
    commit_output = conn.send_command("commit")
    print("\n=== 💾 Commit output ===")
    print(commit_output)

    # Step 6: Verify post-change configuration and show diff
    after_config = conn.send_command("show running-config interface Loopback201")

    print("\n=== 🔍 Post-change diff (before → after) ===")
    post_diff = difflib.unified_diff(
        before_config.splitlines(),
        after_config.splitlines(),
        fromfile="before",
        tofile="after",
        lineterm=""
    )
    post_diff_text = "\n".join(post_diff)
    print(post_diff_text if post_diff_text else "(no differences detected)")

print("\n✅ Done. Loopback201 created and description committed successfully.")

✅ Connected to: RP/0/RP0/CPU0:ios#

=== 🧪 Dry-run diff (before → intended) ===
--- before
+++ intended
@@ -1,3 +1,2 @@
-
-Tue Nov  4 11:48:43.958 UTC
-% No such configuration item(s)
+interface Loopback201
+description TESTDEMO

=== 🚀 Applying candidate configuration ===
configure terminal

Tue Nov  4 11:48:49.980 UTC
RP/0/RP0/CPU0:ios(config)#interface Loopback201

RP/0/RP0/CPU0:ios(config-if)#description TESTDEMO

RP/0/RP0/CPU0:ios(config-if)#

=== 💾 Commit output ===

Tue Nov  4 11:48:51.271 UTC

=== 🔍 Post-change diff (before → after) ===
--- before
+++ after
@@ -1,3 +1,5 @@
 
-Tue Nov  4 11:48:43.958 UTC
-% No such configuration item(s)
+Tue Nov  4 11:48:52.423 UTC
+interface Loopback201
+ description TESTDEMO
+!

✅ Done. Loopback201 created and description committed successfully.


👀 Let's verify if it was indeed created:

In [10]:
with ConnectHandler(**device) as conn:
    print("Connected:", conn.find_prompt())
    int_brief = conn.send_command("show ipv4 interface brief")
    print("\n=== show ipv4 interface brief ===\n", int_brief)

Connected: RP/0/RP0/CPU0:ios#

=== show ipv4 interface brief ===
 
Tue Nov  4 11:49:00.057 UTC

Interface                      IP-Address      Status          Protocol Vrf-Name
Loopback100                    1.1.1.100       Up              Up       default 
Loopback201                    unassigned      Up              Up       default 
Loopback555                    unassigned      Up              Up       default 
MgmtEth0/RP0/CPU0/0            10.10.20.175    Up              Up       default 
GigabitEthernet0/0/0/0         unassigned      Shutdown        Down     default 
GigabitEthernet0/0/0/1         unassigned      Shutdown        Down     default 
GigabitEthernet0/0/0/2         unassigned      Shutdown        Down     default 
GigabitEthernet0/0/0/3         unassigned      Shutdown        Down     default 
GigabitEthernet0/0/0/4         unassigned      Shutdown        Down     default 
GigabitEthernet0/0/0/5         unassigned      Shutdown        Down     default 
GigabitEthern

Let's remove now this configuration:

In [ ]:
# === Target interface to remove ===
interface = "Loopback201"

# === Connect ===
with ConnectHandler(**device) as conn:
    print("✅ Connected to:", conn.find_prompt())

    # Step 1: Capture the current interface configuration
    before_config = conn.send_command(f"show running-config interface {interface}")

    if "interface" not in before_config:
        print(f"⚠️ Interface {interface} does not exist or has no config. Nothing to remove.")
        exit(0)

    # Step 2: Intended removal configuration
    intended_config = f"! Interface {interface} will be deleted"

    # Step 3: Display dry-run diff
    print("\n=== 🧪 Dry-run diff (before → intended removal) ===")
    diff = difflib.unified_diff(
        before_config.splitlines(),
        intended_config.splitlines(),
        fromfile="before",
        tofile="intended",
        lineterm=""
    )
    diff_text = "\n".join(diff)
    print(diff_text if diff_text else "(no differences detected)")

    # Step 4: Apply the removal configuration
    print("\n=== 🧹 Removing interface ===")
    output = conn.send_config_set([f"no interface {interface}"])
    print(output)

    # Step 5: Commit changes
    commit_output = conn.send_command("commit")
    print("\n=== 💾 Commit output ===")
    print(commit_output)

    # Step 6: Verify post-change
    after_config = conn.send_command(f"show running-config interface {interface}")

    print("\n=== 🔍 Post-change diff (before → after) ===")
    post_diff = difflib.unified_diff(
        before_config.splitlines(),
        after_config.splitlines(),
        fromfile="before",
        tofile="after",
        lineterm=""
    )
    post_diff_text = "\n".join(post_diff)
    print(post_diff_text if post_diff_text else f"(Interface {interface} successfully removed.)")

print(f"\n✅ Done. Interface {interface} has been removed and committed.")


✅ Connected to: RP/0/RP0/CPU0:ios#

=== 🧪 Dry-run diff (before → intended removal) ===
--- before
+++ intended
@@ -1,5 +1 @@
-
-Tue Nov  4 11:49:08.186 UTC
-interface Loopback201
- description TESTDEMO
-!
+! Interface Loopback201 will be deleted

=== 🧹 Removing interface ===
configure terminal

Tue Nov  4 11:49:12.795 UTC
RP/0/RP0/CPU0:ios(config)#no interface Loopback201

RP/0/RP0/CPU0:ios(config)#

=== 💾 Commit output ===

Tue Nov  4 11:49:13.887 UTC

=== 🔍 Post-change diff (before → after) ===
--- before
+++ after
@@ -1,5 +1,3 @@
 
-Tue Nov  4 11:49:08.186 UTC
-interface Loopback201
- description TESTDEMO
-!
+Tue Nov  4 11:49:14.946 UTC
+% No such configuration item(s)

✅ Done. Interface Loopback201 has been removed and committed.


👀 It should've been deleted by now:

In [12]:
with ConnectHandler(**device) as conn:
    print("Connected:", conn.find_prompt())
    int_brief = conn.send_command("show ipv4 interface brief")
    print("\n=== show ipv4 interface brief ===\n", int_brief)

Connected: RP/0/RP0/CPU0:ios#

=== show ipv4 interface brief ===
 
Tue Nov  4 11:49:23.382 UTC

Interface                      IP-Address      Status          Protocol Vrf-Name
Loopback100                    1.1.1.100       Up              Up       default 
Loopback555                    unassigned      Up              Up       default 
MgmtEth0/RP0/CPU0/0            10.10.20.175    Up              Up       default 
GigabitEthernet0/0/0/0         unassigned      Shutdown        Down     default 
GigabitEthernet0/0/0/1         unassigned      Shutdown        Down     default 
GigabitEthernet0/0/0/2         unassigned      Shutdown        Down     default 
GigabitEthernet0/0/0/3         unassigned      Shutdown        Down     default 
GigabitEthernet0/0/0/4         unassigned      Shutdown        Down     default 
GigabitEthernet0/0/0/5         unassigned      Shutdown        Down     default 
GigabitEthernet0/0/0/6         unassigned      Shutdown        Down     default 


### 4️⃣ Putting it all together

Now, let's put everything together in a single python script!
- The script `scripts/xr_loopback_set.py` can create loopback interfaces with a number and a description, and also rollback the configurations
- The script can take as arguments either a single line, or a CSV file with multiple interfaces
- The script can also generate a rollback file which can later be used to undo the configurations applied

Let's use the file `scripts/loopbacks.csv` and commit the loopback interfaces, while generating the rollback file `rollback_testdemo.cmds`

In [ ]:
!python3 scripts/xr_loopback_set.py --csv scripts/loopbacks.csv --generate-rollback rollback_testdemo.cmds

Planned commands:
   interface Loopback101
   description Demo Management Loopback
   interface Loopback201
   description Demo Telemetry Loopback
   interface Loopback301
   description Demo Prometheus Exporter
   interface Loopback401
   description Demo IPv6 Tunnel Endpoint
   interface Loopback501
   description Demo Customer Testing Interface

Backup saved: backups/sandbox-iosxr-1.cisco.com-20251104-115157.running-config.txt
Rollback file generated: rollback_20251104.cmds
   no interface Loopback101
   no interface Loopback201
   no interface Loopback301
   no interface Loopback401
   no interface Loopback501

=== device response ===
 configure terminal

Tue Nov  4 12:39:45.152 UTC
RP/0/RP0/CPU0:ios(config)#interface Loopback101

RP/0/RP0/CPU0:ios(config-if)#description Demo Management Loopback

RP/0/RP0/CPU0:ios(config-if)#interface Loopback201

RP/0/RP0/CPU0:ios(config-if)#description Demo Telemetry Loopback

RP/0/RP0/CPU0:ios(config-if)#interface Loopback301

RP/0/RP0/CPU0:ios(

👀 Let's have a look at those interfaces ...

In [26]:
with ConnectHandler(**device) as conn:
    print("Connected:", conn.find_prompt())
    int_brief = conn.send_command("show ipv4 interface brief")
    print("\n=== show ipv4 interface brief ===\n", int_brief)

Connected: RP/0/RP0/CPU0:ios#

=== show ipv4 interface brief ===
 
Tue Nov  4 12:40:35.159 UTC

Interface                      IP-Address      Status          Protocol Vrf-Name
Loopback100                    1.1.1.100       Up              Up       default 
Loopback555                    unassigned      Up              Up       default 
MgmtEth0/RP0/CPU0/0            10.10.20.175    Up              Up       default 
GigabitEthernet0/0/0/0         unassigned      Shutdown        Down     default 
GigabitEthernet0/0/0/1         unassigned      Shutdown        Down     default 
GigabitEthernet0/0/0/2         unassigned      Shutdown        Down     default 
GigabitEthernet0/0/0/3         unassigned      Shutdown        Down     default 
GigabitEthernet0/0/0/4         unassigned      Shutdown        Down     default 
GigabitEthernet0/0/0/5         unassigned      Shutdown        Down     default 
GigabitEthernet0/0/0/6         unassigned      Shutdown        Down     default 


🔥 Nevertheless, let's rollback them all:

In [25]:
!python3 scripts/xr_loopback_set.py --apply-rollback rollback_20251104.cmds

Loaded rollback commands:
   no interface Loopback101
   no interface Loopback201
   no interface Loopback301
   no interface Loopback401
   no interface Loopback501

Backup saved: backups/sandbox-iosxr-1.cisco.com-20251104-115236.running-config.txt

=== device response ===
 configure terminal

Tue Nov  4 12:40:20.697 UTC
RP/0/RP0/CPU0:ios(config)#no interface Loopback101

RP/0/RP0/CPU0:ios(config)#no interface Loopback201

RP/0/RP0/CPU0:ios(config)#no interface Loopback301

RP/0/RP0/CPU0:ios(config)#no interface Loopback401

RP/0/RP0/CPU0:ios(config)#no interface Loopback501

RP/0/RP0/CPU0:ios(config)#

=== commit ===

Tue Nov  4 12:40:22.528 UTC

=== running-config diff (before → after) ===
--- before
+++ after
@@ -1,8 +1,8 @@
 
-Tue Nov  4 12:40:20.098 UTC
+Tue Nov  4 12:40:23.554 UTC
 Building configuration...
 !! IOS XR Configuration 7.3.2
-!! Last configuration change at Tue Nov  4 12:39:47 2025 by admin
+!! Last configuration change at Tue Nov  4 12:40:22 2025 by admin
 !
 snmp-

👀 A final look to what is left:

In [27]:
with ConnectHandler(**device) as conn:
    print("Connected:", conn.find_prompt())
    int_brief = conn.send_command("show ipv4 interface brief")
    print("\n=== show ipv4 interface brief ===\n", int_brief)

Connected: RP/0/RP0/CPU0:ios#

=== show ipv4 interface brief ===
 
Tue Nov  4 12:47:40.062 UTC

Interface                      IP-Address      Status          Protocol Vrf-Name
Loopback100                    1.1.1.100       Up              Up       default 
Loopback555                    unassigned      Up              Up       default 
MgmtEth0/RP0/CPU0/0            10.10.20.175    Up              Up       default 
GigabitEthernet0/0/0/0         unassigned      Shutdown        Down     default 
GigabitEthernet0/0/0/1         unassigned      Shutdown        Down     default 
GigabitEthernet0/0/0/2         unassigned      Shutdown        Down     default 
GigabitEthernet0/0/0/3         unassigned      Shutdown        Down     default 
GigabitEthernet0/0/0/4         unassigned      Shutdown        Down     default 
GigabitEthernet0/0/0/5         unassigned      Shutdown        Down     default 
GigabitEthernet0/0/0/6         unassigned      Shutdown        Down     default 
